# Project 2b - Projectile Motion : Improved Integrators and a unified interface

## Due 2/13

We have now seen a few different versions of the particle motion problem: one for 1D motion without drag, one for 1D motion with drag, and then an extension to 2D motion with drag.  All of these equations have something in common, which is that they can be written in the form
$$
\frac{d \mathbf{u}}{d t} = \mathcal{F}(t,\mathbf{u})
$$
with $\mathbf{u}$ being a vector of state variables (for example, position and velocity in one or two or more directions), and $\mathcal{F}$ representing the right hand sides of the equations of motion.  For example in 2D drag, we might write
$$
\mathbf{u} = \begin{bmatrix} x \\ v_x \\ z \\ v_z \end{bmatrix}
$$
and 
$$
\mathcal{F}(t,\mathbf{u}) = \begin{bmatrix} v_x \\ 
                                        -\frac{c_d}{m} \sqrt{v_x^2 + v_z^2} \;v_x  \\
                                        v_z \\
                                        g -\frac{c_d}{m} \sqrt{v_x^2 + v_z^2} \;v_z \end{bmatrix}.
$$
Indeed, many important scenarios that we may wish to model, ranging from Newtonian physics to population dynamics and beyond fall within this framework.  Because we will be modelling many such systems, it will be helpful to develop a convenient framework for operating on such systems.  In particular, we would like to develop an object oriented framework for working with these problems so that we can reuse as much code as possible (while also keeping our code nice).  We will continue to modify these methods as we require additional functionality, but these will form a good start.  

### A class for problem specification
Our first task will be to create a python class representing our problem.  This doesn't really need to do much: all it really needs to return is the function $\mathcal{F}(t,\mathbf{u})$ and to store any information relevant to the class.  As an example, I've provided a class for the drag-free 1D particle motion equation.  **Create a new class using the existing one as a template that implements the 2D equations with drag.**  

In [7]:
import numpy as np


class ParticleMotion1D:
    """ This is an example class for an ODE specification"""
    
    def __init__(self,g=-9.81):
        
        self.n_dof = 2
        self.g = g
        
    def rhs(self,t,u):
        # the right hand side of the ode (or $\mathcal{F}(t,u)$)
        dudt = np.zeros(self.n_dof)
        dudt[0] = u[1]
        dudt[1] = self.g
        return dudt
    
class ParticleMotion2DWithDrag:
    
    # Do you need other keyword arguments here?
    # we will need: g, c, m
    def __init__(self,g=-9.81,c=1,m=1):
        self.n_dof = 4
        self.g = g
        self.c = c
        self.m = m
    
    def rhs(self,t,u):
        pass
    

### A class for time stepping
We will also be utilizing multiple different methods for performing numerical integration.  One simple choice is Euler's method.  A reasonable class for doing this might be



In [8]:
class Euler:
    def __init__(self):
        pass   
    
    def step(self,ode,t,dt,u_0):
        u_1 = u_0 + dt*ode.rhs(t,u_0)
        return u_1

Note that there is no reference to particle motion or to any other particular problem specifically: all this requires is the current time, the desired time step, the initial state, and the object representing the problem.  Thus, one could easily use either my implementation of the simple 1D particle or your implementation of the 2D particle *with no modification to the code* (assuming that shapes are all correct, which is something one might wish to check).  This modularity is, obviously, highly desirable.  We can also view this class as a template for more performant time-stepping schemes such as the Runge-Kutta methods.  Heun's method (a variant on Runge-Kutta 2) is defined as 
$$
\mathbf{u}_{n+1} = \mathbf{u}_n + \frac{\Delta t}{2}(k_1 + k_2)
$$
$$
k_1 = \mathcal{F}(t,\mathbf{u}_n)
$$
$$
k_2 = \mathcal{F}(t + \Delta t, \mathbf{u}_n + \Delta t k_1)
$$
**Adapt the above class to implement Heun's method**



In [9]:
class Heun:
    def __init__(self):
        pass   
    
    def step(self,ode,t,dt,u_0):
        # Do some stuff here
        return u_1

**Also implement RK4** (Gould Eqs. 3.59, 3.60)

In [10]:
class RK4:
    def __init__(self):
        pass   
    
    def step(self,ode,t,dt,u_0):
        # Do some stuff here
        return u_1

Finally, the typical use case for these objects is to perform an integration over multiple time steps, and its nice to contain such operations in a class.  A reasonable implementation of such a thing might look like:

In [11]:
class Integrator:
    def __init__(self,ode,method): # ode is provided as one of the ParticleMotion classes, method is one of the integrator classes
        self.ode = ode # store the ode
        self.method = method # store the method
        
    def integrate(self,interval,dt,u_0): # interval is a list [t_0,t_end], dt is the time step, u_0 is the initial state
        t_0 = interval[0] # initial time
        t_end = interval[1] # final time
        
        times = [t_0] # list to store the times
        states = [u_0] # list to store the states
        
        t = t_0 # current time
        while t<t_end: # loop over the time interval
            dt_ = min(dt,t_end-t) # time step
            u_1 = self.method.step(self.ode,t,dt_,u_0) # integrate the ODE
            t = t + dt_ # update the time
            u_0 = u_1 # update the state
            
            times.append(t) # store the time
            states.append(u_1) # store the state
            
        return np.array(times),np.array(states) # return the times and states as numpy arrays

No modifications to the above code are needed at this stage, but I would like you to **go through it, line by line, and add comments describing what all of the elements are doing**.  This should be straightforward, as I hope that it looks very much like some code that you've already written.  

An example of all of these components working together is

In [12]:
pm = ParticleMotion1D()
method = Euler()
integrator = Integrator(pm,method)

t_0 = 0.0
t_end = 1.0
dt = 0.01
z_0 = 5.0
v_0 = 5.0
t,u = integrator.integrate([t_0,t_end],dt,np.array([z_0,v_0]))

plt.plot(t,u[:,0])

NameError: name 'plt' is not defined

**Verify your implementations by testing similar code to the above block with all different combinations of problems and time stepping schemes.**  Note that if your improved time-stepping schemes are working, a particle integrated using RK2 or RK4 with relatively large time steps (say $\Delta t=0.1$ should end up in a very similar place to one integrated with Euler's method with a very small time step (which is implemented correctly) (say $\Delta t=.001$).